In [1]:
import pandas as pd
import numpy as np

In [2]:
def wrangle():
    
    """
    
    data: Initial Dataframe
    interim: Single Month Dataframe
    df: final dataframe (concatenated)
    """
    
   
    # Empty Final Dataframe
    df = pd.DataFrame()
    
    # Importing URL's from Text File
    urls = pd.read_csv('./data/to_download.txt', 
                   sep=" ",
                   header=None)
    
    # Loops through URLS. 
    for url in urls.sort_values(by=0)[0].values:
        # Import Data
        data = pd.read_csv(url)

        # Empty Dataframes
        interim = pd.DataFrame()


        ########### Datetime Manipulations ###########

        # Convert to Datetime
        data['tpep_pickup_datetime'] = pd.to_datetime(data['tpep_pickup_datetime'])
        data['tpep_dropoff_datetime'] = pd.to_datetime(data['tpep_dropoff_datetime'])

        # Ride Length
        data['ride_length'] = ((data['tpep_dropoff_datetime'] - data['tpep_pickup_datetime'])
                                .apply(lambda x:x.total_seconds()))
        # Pickup Day for Groupby function
        data['pickup_day'] = data['tpep_pickup_datetime'].map(lambda x:x.strftime('%d'))


        ########### Continuous Variables ###########

        cont_vars = ['passenger_count','trip_distance','fare_amount','extra',
                     'mta_tax','tip_amount','tolls_amount','total_amount','ride_length']


        for calc in ['mean','median','mode']:
            for cont_var in cont_vars:
                # Mean
                if calc == 'mean':
                    interim[cont_var+'_'+calc] = data.groupby('pickup_day')[cont_var].mean()
                # Median
                elif calc == 'median':
                    interim[cont_var+'_'+calc] = data.groupby('pickup_day')[cont_var].median()
                # Mode
                # Pandas Open Source: Please add a Mode Aggregation Function.
                # value_counts is not an adequate replacement for a vectorized function
                else:
                    modes = []
                    for i in np.sort(data['pickup_day'].unique()):
                        mode_test = data.groupby('pickup_day')[cont_var].value_counts()
                        modes.append(mode_test.loc[i].values[0])
                    interim[cont_var+'_'+calc] = modes


        ########### Discrete Variables ###########

        cat_var = ['RatecodeID','store_and_fwd_flag','payment_type','improvement_surcharge']

        for cat in cat_var:
            #print(cat)
            one_hot = pd.get_dummies(data[cat])
            one_hot_columns = one_hot.columns
            one_hot = pd.concat([data['pickup_day'],one_hot], axis=1)

            for column in one_hot_columns:
                interim[cat+'_'+str(column)] = one_hot.groupby('pickup_day')[column].sum()


        ########### Location ID Data ###########


        for loc in ['PULocationID','DOLocationID']:

            first_loc  = []
            second_loc = []
            third_loc  = []
            fourth_loc = []
            fifth_loc  = []

            for day in np.sort(data['pickup_day'].unique()):
                per_day = (data.groupby('pickup_day')[loc]
                           .value_counts()[day]
                           .iloc[0:5]
                           .index
                           .tolist())

                first_loc.append(per_day[0])
                second_loc.append(per_day[1])
                third_loc.append(per_day[2]) 
                fourth_loc.append(per_day[3])
                fifth_loc.append(per_day[4])

            interim[loc+'_'+'1'] = first_loc
            interim[loc+'_'+'2'] = second_loc
            interim[loc+'_'+'3'] = third_loc
            interim[loc+'_'+'4'] = fourth_loc
            interim[loc+'_'+'5'] = fifth_loc

                                                              
        ########### Create Date Column ###########
                                                              
        date = []

        for i in interim.index:
            date.append(url[-11:-4]+'-'+interim.index[0])
        interim['date'] = date
                                                              
        ########### Concatenate Dataframes ###########                                                     

        df = pd.concat([df,interim],axis=0,ignore_index=True, sort=False)
        
        print('##########', url[-11:-4], 'Added ##########')
    
    return df

In [3]:
%%time
df = wrangle()

########## 2017-01 Added ##########
########## 2017-02 Added ##########
########## 2017-03 Added ##########
########## 2017-04 Added ##########
########## 2017-05 Added ##########
########## 2017-06 Added ##########
########## 2017-07 Added ##########
########## 2017-08 Added ##########
########## 2017-09 Added ##########
########## 2017-10 Added ##########
########## 2017-11 Added ##########
########## 2017-12 Added ##########
########## 2018-01 Added ##########
########## 2018-02 Added ##########
########## 2018-03 Added ##########
########## 2018-04 Added ##########
########## 2018-05 Added ##########
########## 2018-06 Added ##########
########## 2018-07 Added ##########
########## 2018-08 Added ##########
########## 2018-09 Added ##########
########## 2018-10 Added ##########
########## 2018-11 Added ##########
########## 2018-12 Added ##########
CPU times: user 6h 7min 42s, sys: 21min 59s, total: 6h 29min 41s
Wall time: 6h 6min 48s


In [5]:
df.to_csv('2017-2018_grouped_tax.csv')

## Clustering

In [34]:
df = pd.read_csv('./data/yellow_tripdata_2016-10.csv')

In [14]:
test = 'https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2015-10.csv'[-11:-7]

In [18]:
if int(test) > 2015:
    print('yay')

In [46]:
data.shape

(10854626, 17)

In [48]:
data.append(data).shape

(21709252, 17)

In [259]:
'yellow_tripdata_2018-01.csv'[-11:-4]+'-'+data.index[0]

'2018-01-01'

In [28]:
df.tail()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,pickup_date
8759869,1,2018-01-31 23:21:35,2018-01-31 23:34:20,2,2.80,1,N,158,163,1,12.0,0.5,0.5,2.65,0.0,0.3,15.95,2018-01-31
8759870,1,2018-01-31 23:35:51,2018-01-31 23:38:57,1,0.60,1,N,163,162,1,4.5,0.5,0.5,1.15,0.0,0.3,6.95,2018-01-31
8759871,2,2018-01-31 23:28:00,2018-01-31 23:37:09,1,2.95,1,N,74,69,2,10.5,0.5,0.5,0.00,0.0,0.3,11.80,2018-01-31
8759872,2,2018-01-31 23:24:40,2018-01-31 23:25:28,1,0.00,1,N,7,193,2,0.0,0.0,0.0,0.00,0.0,0.0,0.00,2018-01-31
8759873,2,2018-01-31 23:28:16,2018-01-31 23:28:38,1,0.00,1,N,7,193,2,0.0,0.0,0.0,0.00,0.0,0.0,0.00,2018-01-31


In [6]:
df.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount'],
      dtype='object')

In [251]:
    
for loc in ['PULocationID','DOLocationID']:
    
    first_loc  = []
    second_loc = []
    third_loc  = []
    fourth_loc = []
    fifth_loc  = []

    for day in np.sort(df['pickup_day'].unique()):
        per_day = (df.groupby('pickup_day')[loc]
                   .value_counts()[day]
                   .iloc[0:5]
                   .index
                   .tolist())

        first_loc.append(per_day[0])
        second_loc.append(per_day[1])
        third_loc.append(per_day[2]) 
        fourth_loc.append(per_day[3])
        fifth_loc.append(per_day[4])

    data[loc+'_'+'1'] = first_loc
    data[loc+'_'+'2'] = second_loc
    data[loc+'_'+'3'] = third_loc
    data[loc+'_'+'4'] = fourth_loc
    data[loc+'_'+'5'] = fifth_loc
    
data

,passenger_count_mean,trip_distance_mean,fare_amount_mean,extra_mean,mta_tax_mean,tip_amount_mean,tolls_amount_mean,total_amount_mean,passenger_count_median,trip_distance_median,...,PULocationID_1,PULocationID_2,PULocationID_3,PULocationID_4,PULocationID_5,DOLocationID_1,DOLocationID_2,DOLocationID_3,DOLocationID_4,DOLocationID_5
pickup_day,,,,,,,,,,,,,,,,,,,,,
01,1.713753,3.365019,13.186019,0.230123,0.495671,1.652965,0.373681,16.244469,1,1.80,...,186,48,230,79,161,48,170,186,230,79
02,1.631086,3.025751,12.445166,0.392684,0.497153,1.658423,0.381970,15.678489,1,1.50,...,161,186,236,230,237,161,236,237,230,162
03,1.601184,2.792630,12.236337,0.392187,0.497621,1.677209,0.313631,15.419717,1,1.50,...,161,237,162,236,186,161,236,237,162,230
04,1.629486,2.576409,11.492634,0.373150,0.496703,1.537451,0.195229,14.403393,1,1.43,...,230,161,48,162,186,161,230,162,236,237
05,1.626063,2.592781,12.066120,0.409144,0.497481,1.707829,0.264447,15.252301,1,1.42,...,236,237,161,230,162,161,236,237,230,170
06,1.671101,2.757388,12.018641,0.163535,0.497371,1.630359,0.243952,14.858063,1,1.50,...,79,230,142,236,237,236,79,142,48,230
07,1.659219,3.000356,12.299799,0.150281,0.496935,1.727601,0.337075,15.314979,1,1.58,...,79,236,230,132,142,236,48,142,237,79
08,1.575695,2.939049,13.038814,0.379614,0.497469,1.910664,0.364657,16.495807,1,1.50,...,236,161,237,162,132,236,161,237,170,162
09,1.578616,2.789331,12.653291,0.386606,0.497787,1.903532,0.313238,16.057577,1,1.56,...,237,236,161,162,186,236,161,237,162,170


In [246]:
df.groupby('pickup_day')['PULocationID'].value_counts()['01'].iloc[0:5].index.tolist()

[186, 48, 230, 79, 161]

In [226]:
pd.concat([df['pickup_day'],one_hot], axis=1)

pickup_day    0
1             0
2             0
3             0
4             0
5             0
6             0
99            0
dtype: int64

In [161]:
oh_cat = df.groupby('pickup_day')['RatecodeID'].value_counts()

In [250]:
df.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'pickup_date', 'pickup_day', 'ride_length',
       'passenger_countmean', 'trip_distancemean', 'fare_amountmean',
       'extramean', 'mta_taxmean', 'tip_amountmean', 'tolls_amountmean',
       'total_amountmean', 'passenger_countmedian', 'trip_distancemedian',
       'fare_amountmedian', 'extramedian', 'mta_taxmedian', 'tip_amountmedian',
       'tolls_amountmedian', 'total_amountmedian'],
      dtype='object')

In [188]:
enc = OneHotEncoder()

x = enc.fit_transform(df[cat_var])

In [191]:
(x.toarray()).shape

(8759874, 17)

In [208]:
pd.get_dummies(df['store_and_fwd_flag'])

Index(['N', 'Y'], dtype='object')

In [159]:
df['RatecodeID'].unique()

for var in cat_var:
    for val in df[var].unique()
    

array([ 1,  2,  5,  4,  3,  6, 99])

In [151]:
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])
df['ride_length'] =(df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).apply(lambda x:x.total_seconds())
df['pickup_day'] = df['tpep_pickup_datetime'].map(lambda x:x.strftime('%d'))

In [135]:
mode_index

Index(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24',
       '25', '26', '27', '28', '29', '30', '31'],
      dtype='object', name='pickup_day')

In [102]:
data = pd.DataFrame()